# **Webscraper für die Zeitungsliste von DigiPress**

## Vorbereitung

In [ ]:
#Installation benötigter Python-Bibliotheken

import requests # Bibliothek zur Kommunikation über HTTP (Hypertext Transfer Protocol)
from bs4 import BeautifulSoup # Bibliothek für Webcrawling/Screenscraping, Parsen von HTML und XML
import pandas as pd # Bibliothek zur Datenverarbeitung, -analyse und -darstellung

## HTML der Zeitungsliste laden

In [ ]:
# URL der Website
url = "https://digipress.digitale-sammlungen.de/titles"

# HTTP-Anfrage über Requests = Sammeln des HTMLs
response = requests.get(url)
response.raise_for_status()  # Fehlermeldung bei Problemen

# HTML der Website parsen = BeautifulSoup liest den von Requests erhaltenen Output als HTML ein
soup = BeautifulSoup(response.text, 'html.parser')

# HTML der Website zur Kontrolle ausgeben
print(soup)

## HTML parsen und Metadaten extrahieren

In [ ]:
# Leere Liste für zu sammelnde Informationen anlegen
data = []

# Für jedes div-Element der Klasse "seriesItem" (Suche über .find_all-Funktion von Beautiful Soup)
for series_item in soup.find_all('div', class_='seriesItem'):

    # Platzhalter erstellen (= zu befüllende Spalten)
    title = None
    digipress_id = None
    zeitungsunternehmen = None

    #Teil 1: Verarbeitung von Überschriften (<h4>)

    # Suche nach <h4>-Elementen (sollte in jedem "seriesItem" vorhanden sein, Suche über .find-Funktion von Beautiful Soup)
    h4 = series_item.find('h4')

    # Fall 1: <h4> mit name="digiPressID" - Beispiel: Aarauer Zeitung
    if h4 and h4.has_attr('name'): # Falls <h4>-Element mit "name"-Attribut gefunden:

        # digiPress-ID (digipress_id) identifizieren
        digipress_id = h4['name'] # Aus Attribut extrahieren

        # Zeitungstitel (title) identifizieren
        title_element = h4.find('a', class_='newspaperTitle') # Finde <a>-Element mit Klasse "newspaperTitle"
        if title_element: #Wenn gefunden:
          title = title_element.text.strip() #Text in Element von vor- oder nachgestelltem Whitespace bereinigen (.strip()-Funktion) und als Titel speichern

    # Fall 2: <h4> mit class="noLink" - Beispiel:
    elif h4 and 'noLink' in h4.get('class', []): # Elif = else if, d.h. ansonsten: Falls <h4>-Element gefunden und Klasse = "noLink":

        # Zeitungstitel (title) identifizieren
        span = h4.find('span', recursive=False)  # Finde das erste <span>-Element ohne Attribute
        if span: # Falls gefunden:
          title = span.text.strip() # Text bereinigen und als Titel speichern

        # Zeitungsunternehmen (zeitungsunternehmen) und digiPress-ID (id)
        link_element = series_item.find('a', class_='smaller linkToPrimaryTitle') # Finde <a>-Element mit Klasse "smaller linktoPrimaryTitle"
        if link_element: # Falls gefunden:

            #zeitungsunternehmen
            zeitungsunternehmen = link_element.text.strip() # Bereinigten Text in Linkelement als Zeitungsunternehmen speichern

            # digiPress-id (digipress_id)
            if link_element.has_attr('href'): # Falls Element Attribut "href" hat:
              digipress_id = link_element['href'].strip("#") # URL aus Attribut extrahieren - "#" entfernen

    # Teil 2: Verarbeitung von Metadaten (<dl>)

    # Leeres Dictionary (= Liste von Paaren aus Schlüssel und Wert) für Metadaten anlegen
    metadata = {}

    # Finde <dl>-Element mit der Klasse "seriesItemMeta"
    meta_container = series_item.find('dl', class_='seriesItemMeta')

    if meta_container: # Falls gefunden:
        for dt, dd in zip(meta_container.find_all('dt'), meta_container.find_all('dd')): # Für alle <dt> und <dd>-Elemente (je durch zip()-Funktion zu Paar verbunden):
            metadata[dt.text.strip()] = dd.text.strip() # Daten speichern: dt = Schlüssel (z.B. "Erscheinungsverlauf"), dd = Wert (z.B. "1815-1821")

    # Einen Eintrag pro Zeitungstitel (im Sinne von <div>-Element) anhand der Informationen aus <h4> erstellen
    entry = {
        "Titel": title,
        "digiPress-ID": digipress_id,
        "Unternehmen": zeitungsunternehmen,
    }
    entry.update(metadata) # Metadaten aus <dl> hinzufügen
    data.append(entry) # Eintrag zu Gesamtdaten hinzufügen

# Gesamtdaten in Pandas DataFrame umwandeln
df = pd.DataFrame(data)

# DataFrame anzeigen
df

## Ergänzung der Daten

**Frage:** Welche wichtigen Informationen fehlen noch?

**Aufgabe:** Welcher Wert gehört in die Spalte "Unternehmen", wenn None statt einem tatsächlichen Namen gegeben ist? Welche Spalte muss in der Code-Zelle unterhalb eingesetzt werden?

In [ ]:
# Ergänzung von Namen des Zeitungsunternehmen, wo None

df.loc[df["Unternehmen"].isna(), "Unternehmen"] = df["???"] # Für Zeilen, wo "Unternehmen" leer ist, soll Wert der Spalte ??? übernommen werden
df

In [ ]:
# Link ergänzen

base_url = "https://digipress.digitale-sammlungen.de/calendar/newspaper/" # Basis-URL für die Kalenderansicht: DigiPress-ID muss je am Ende hinzugefügt werden
df["Link"] = "https://digipress.digitale-sammlungen.de/calendar/newspaper/" + df["digiPress-ID"]
df.head()

## Genauerer Blick in die Daten

### Wie viele Einträge umfasst die Zeitungsliste überhaupt?

**Erinnerung:** Zeitungsunternehmen (mit unterschiedlicher digiPress-ID) versus Zeitungstitel (mit unterschiedlicher ZDB-ID)

In [ ]:
# Wie viele unterschiedliche Zeitungstitel (= ZDB-IDs) umfasst die Zeitungsliste?

count = df['ZDB-ID'].nunique()
print(f"Anzahl an Zeitungstitel: {count}")

In [ ]:
# Wie viele unterschiedliche Zeitungsunternehmen (= digiPress-ID) sind gelistet?

count = df['digiPress-ID'].nunique()
print(f"Anzahl an Zeitungsunternehmen: {count}")

### Welche Spalten sind für die Analyse von Interesse?

In [ ]:
# Wie viele Titel haben Informationen einer bestimmten Metadaten-Kategorie oder Spalte ("z.B. Paralleltitel")?

count = df['Paralleltitel'].notna().sum() #Zähle die Werte, die nicht leer sind
print(count)

In [ ]:
# Wie viele unterschiedliche Werte hat eine bestimmte Spalte?

print(len(df["Paralleltitel"].unique())) # Gib die Anzahl einzigartiger Werte einer Spalte aus (len() ermittelt Länge einer Liste)

In [ ]:
# Wie sehen die Werte einer bestimmten Spalte aus?

print(df["Paralleltitel"].unique()) # Gib die einzigartigen Werte einer Spalte aus

## Nicht benötigte Spalten für besseren Überblick entfernen

In [ ]:
delete_list = ["Paralleltitel", "Unterreihe", "Beteiligt"] # Liste an zu löschenden Spalten
df = df.drop(columns=delete_list)
df.head()

## Speicherung als Excel

In [ ]:
df.to_excel("Zeitungsliste.xlsx", index=False)

## Nächste Schritte

* **Für nähere Analyse der zeitlichen Verteilung:** Notebook "Analyse_Zeitliche_Verteilung.ipynb"

* **Für nähere Analyse der räumlichen Verteilung:** Notebook "Analyse_Räumliche_Verteilung.ipynb"